### Classes

Finally we get to classes.

I assume you already have some knowledge of classes and OOP in general, so I'll focus on the semantics of creating classes and some of the differences with Java classes.

First, the question of visibility. There is no such thing as private or public in Python. Everything is public, period. So we don't have to specify the visibility of functions and attributes in Python.

Class instantiations are done in two steps - the instance is created first, and then the instance is initialized. In general we hook into the initialization phase and leave the object creation alone. We do this by using a special method in the class called `__init__`. We'll see how large a role special methods play in Python.

The important thing to note is that by the time `__init__` is called in our class, the object (instance) has **already** been created.

We use the `class` keyword to create classes:

In [1]:
class Person:
    pass

`pass` is something we can use in Python to indicate "do nothing" (a so called "no-op" operation). Here I use it to supply a body for the class definition, but don't actually want to specify any functionality.

So now we can actually create instances of the `Person` object at this point. They will be pretty useless since we have not implemented any functionality yet.

We create instances of classes by **calling** the class - remember how we call things in Python, we use `()`:

In [2]:
p = Person()

In [3]:
id(p), type(p)

(140289742231488, __main__.Person)

So now we may want to add some functions to the class. Whenever we define a function in a class, we have to understand what happens when we call that function from an instance, using dot notation:

For example, if we write 

```p.say_hello()```

then we are calling the `say_hello()` function from the instance, and Python will **bind** that function to the specific instance - i.e. it creates an association between the instance used to call the function, and the function. 

The way it does this is by passing in the instance reference to the function as the first positional argument - in this case it would actually call `say_hello(p)`. And our `say_hello` function now has access to the instance it was called from, including any internal state.

When functions are bound to an instance, they are called **methods** of the class, and, in particular **instance methods** because they are bound to instances of the class when called. (There are other types of functions that can be bound to the class, called *class methods*, but this is beyond the scope of this primer).

Let's see how this works:

In [4]:
class Person:
    def say_hello(instance, name):
        return f'{instance} says hello to {name}'

So here, we had to make sure the first argument of our function was created to receive the instance it is being called from. After that we are free to add our own arguments as needed.

In [5]:
p = Person()

Let's see what `p` looks like when we print it:

In [6]:
p

And now let's call the `say_hello` method from the instance `p`:

In [7]:
p.say_hello('Alex')

'<__main__.Person object at 0x7f97b098e250> says hello to Alex'

You'll notice that we did not pass `p` as the first argument to `say_hello` - Python did that for us since we wrote `p.say_hello`.

By convention, that `instance` argument I wrote above, is usually named `self`, for obvious reasons. But it is just a convention, although one you should stick to.

In [8]:
class Person:
    def say_hello(self, name):
        return f'{self} says hello to {name}'

In [9]:
p = Person()
p.say_hello('Alex')

'<__main__.Person object at 0x7f97b098eaf0> says hello to Alex'

So now let's turn our attention to instance attributes. 

Python is dynamic, so instance attributes do not have to be defined at the time the class is created. In fact we rarely do so.

Let's see how we can add an attribute to an instance after it's been created:

In [10]:
p = Person()

In [11]:
p.name = 'Alex'

That's it, `p` now has an attribute called `name`:

In [12]:
p.name

'Alex'

But this is specific to this instance, not other instances:

In [13]:
p2 = Person()
p2.name

AttributeError: 'Person' object has no attribute 'name'

So instance attributes are **specific** to the instance (hence the name).

Of course we can define attributes by calling methods in our class - let's see an example of this:

In [14]:
class Person:
    def set_name(self, name):
        self.name = name
        
    def get_name(self):
        return self.name

In [15]:
p = Person()

At this point `p` does **not** have a `name` attribute (it hasn't been set yet!):

In [16]:
p.get_name()

AttributeError: 'Person' object has no attribute 'name'

But we can easily set it using the `set_name` method:

In [17]:
p.set_name('Alex')

In [18]:
p.get_name()

'Alex'

And of course the attribute is called `name` and is easily accessible directly as well:

In [19]:
p.name

'Alex'

This is what is called a *bare* attribute - it is not hidden by getter and setter methods like we would normally do in Java (remember we do not have private variables).

You'll notice the issue we had - we would get an exception if we tried to access the attribute before it was actually created.

For this reason, best practice is to create these instance attributes (even setting them to a default value or `None`) when the class instance is being created.

The best place to do this is in the *initialization* phase of the class (remember that class instantiation has two phases - creation and initialization).

To do this we use the special method `__init__`. 

This is going to be a function in our class, and will be bound to the instance when it is called (by that time the instance has already been created), so just like our `set_name` method, we'll need to allow for the instance to be received as the first argument:

In [20]:
class Person:
    def __init__(self, name):
        self.name = name

So the `__init__` method is basically doing the same thing as our `set_name` method - the difference is in how it is called.

When we create an instance using `Person()`, Python looks for, and if available calls, the `__init__` method (that's why it's called a *special method*) **after** the class instance has been created (hence the name `__init__` - it is the initialization phase, that occurs after the creation phase).

In our case here, the first argument will receive the just created object, but we have one additional argument, `name`. So we need to pass that in when we create the instance:

In [21]:
p = Person('name')

The `__init__` method was actually called - let's see this:

In [22]:
class Person:
    def __init__(self, name):
        print(f'__init__ called for object {self}')
        self.name = name

In [23]:
p = Person('Alex')

__init__ called for object <__main__.Person object at 0x7f97d046e700>


And in fact, the memory address of `p` is:

In [24]:
hex(id(p))

'0x7f97d046e700'

which as you can see, is exactly the same object that `self` was set to when `__init__` was called.

And our instance `p` now has an attribute called `name`:

In [25]:
p.name

'Alex'

We can create another instance:

In [26]:
p2 = Person('Eric')

__init__ called for object <__main__.Person object at 0x7f97b099ae20>


In [27]:
hex(id(p2)), p2.name

('0x7f97b099ae20', 'Eric')

And that has not affected the `name` of `p` - since `name` is an instance attribute (it is specific to the instance):

In [28]:
p.name

'Alex'